In [66]:
from innolux import *

In [67]:
mgd_db = pl.read_excel(data_path / "材料大地圖" / "demo.xlsx", sheet_name="mgd_db")
isd_attribute_db = pl.read_excel(data_path / "材料大地圖" / "demo.xlsx", sheet_name="isd_attribute_db")
combo_db = pl.read_excel(data_path / "材料大地圖" / "combo_db_20250911.xlsx")

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 11, falling back to string


In [37]:
mgd_db

GROUP_ID,COMBO_SYS_ID,ISD_STATE_KEY,APPLICATION,液晶MODE,MASK_ID,搭配資料ID,項目ID,項目名稱,項目資料ID,資料名稱,FAB
str,str,str,str,str,str,str,str,str,str,str,str
"""MIG_000001""","""1901_00003""","""ACTIVE""","""AA_OEM""","""NOT_USE (無液晶)""",null,"""MGD_000001""","""ITM_000061""","""PI 型號""","""NONE""","""NONE(無製程)""",null
"""MIG_000001""","""1901_00003""","""ACTIVE""","""AA_OEM""","""NOT_USE (無液晶)""",null,"""MGD_000001""","""ITM_000062""","""毛輪布型號""","""NONE""","""NONE(無製程)""",null
"""MIG_000001""","""1901_00003""","""ACTIVE""","""AA_OEM""","""NOT_USE (無液晶)""",null,"""MGD_000001""","""ITM_000064""","""LC 型號""","""NONE""","""NONE(無製程)""",null
"""MIG_000001""","""1901_00003""","""ACTIVE""","""AA_OEM""","""NOT_USE (無液晶)""",null,"""MGD_000001""","""ITM_000071""","""SEAL 型號""","""NONE""","""NONE(無製程)""",null
"""MIG_000001""","""1901_00016""","""ACTIVE""","""NB""","""AAS""",null,"""MGD_000024""","""ITM_000061""","""PI 型號""","""ISD_000425""","""SE-6414""",null
…,…,…,…,…,…,…,…,…,…,…,…
"""MIG_000001""","""2403_00023""","""ACTIVE""","""NB""","""PA_AAS""","""G034 B版光罩""","""MGD_001299""","""ITM_000071""","""SEAL 型號""","""ISD_018690""","""S-WB141(318B)""","""LCD-T6"""
"""MIG_000001""","""2412_00013""","""ACTIVE""","""NB""","""ECB""","""E086""","""MGD_001302""","""ITM_000061""","""PI 型號""","""ISD_000423""","""SE-6414""","""LCD-03"""
"""MIG_000001""","""2412_00013""","""ACTIVE""","""NB""","""ECB""","""E086""","""MGD_001302""","""ITM_000062""","""毛輪布型號""","""ISD_011099""","""Y20-130""","""LCD-03"""


In [47]:
(
    mgd_db.filter(
        # pl.col("資料名稱") == 'LCT-15-1098',
        # pl.col("搭配資料ID").is_in(["MGD_000434", "MGD_000737", "MGD_000863"])
        # pl.col("FAB").is_not_null(),
    )
    .select(
        "搭配資料ID",
        "FAB",
        "APPLICATION",
        "項目ID",
        "項目名稱",
        "項目資料ID",
        "資料名稱",
    )
    .pivot(
        index=["搭配資料ID", "FAB", "APPLICATION"],
        on=["項目名稱"],
        values="資料名稱",
        aggregate_function="first"
    )
    .with_columns(
        pl.lit("RELEASED").alias("PI Phase"),
        pl.lit("RELEASED").alias("LC Phase"),
        pl.lit("RELEASED").alias("SEAL Phase"),
    )
    .select(
        "搭配資料ID",
        pl.col("FAB").alias("使用廠區"),
        pl.col("APPLICATION").alias("應用別"),
        "PI 型號",
        "PI Phase",
        "LC 型號",
        "LC Phase",
        "SEAL 型號",
        "SEAL Phase",
    )
)

搭配資料ID,使用廠區,應用別,PI 型號,PI Phase,LC 型號,LC Phase,SEAL 型號,SEAL Phase
str,str,str,str,str,str,str,str,str
"""MGD_000001""",null,"""AA_OEM""","""NONE(無製程)""","""RELEASED""","""NONE(無製程)""","""RELEASED""","""NONE(無製程)""","""RELEASED"""
"""MGD_000024""",null,"""NB""","""SE-6414""","""RELEASED""","""ZCM-5522XX""","""RELEASED""","""W/R 723K1M""","""RELEASED"""
"""MGD_000054""",null,"""Medical""","""SE-6414""","""RELEASED""","""LCT-09-1060""","""RELEASED""","""W/R 723K1M""","""RELEASED"""
"""MGD_000066""",null,"""NB""","""SE-6414""","""RELEASED""","""LCT-11-2596""","""RELEASED""","""W/R 723K1M""","""RELEASED"""
"""MGD_000071""",null,"""CE""","""SE-5811""","""RELEASED""","""LCT-11-362""","""RELEASED""","""S-WH18""","""RELEASED"""
…,…,…,…,…,…,…,…,…
"""MGD_001293""","""LCD-T2""","""NB""","""AL7171""","""RELEASED""","""ZCM-5522XX""","""RELEASED""","""W/R 723K1M""","""RELEASED"""
"""MGD_001294""","""LCD-T2""","""NB""","""AL7171""","""RELEASED""","""ZCM-5522XX""","""RELEASED""","""S-WB141(318B)""","""RELEASED"""
"""MGD_001298""","""LCD-T6""","""NB""","""NRB-U238(5521)_RB005""","""RELEASED""","""LCT-13-1386""","""RELEASED""","""S-WB141(318B)""","""RELEASED"""


In [41]:
combo_db.head(1)

COMBOID,STATE_KEY,MASKID,MODELNAME,ITEMS_PHASE_KEY,APPLICATION,CF_FAB,LCD_FAB,LC_MODE,ITM_000061_DATA,ITM_000061_PHASE,ITM_000064_DATA,ITM_000064_PHASE,ITM_000050_DATA,ITM_000050_PHASE,ITM_000071_DATA,ITM_000071_PHASE,ITM_000062_DATA,ITM_000062_PHASE
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""COMBOID""","""STATE_KEY""","""MASKID""","""MODELNAME""","""Overall_Phase""","""APPLICATION""","""CF_FAB""","""LCD_FAB""","""LC_MODE""","""PI型號""","""PI_Phase""","""LC型號""","""LC_Phase""","""OC型號""","""OC_Phase""","""SEAL型號""","""SEAL_Phase""","""RubCloth型號""","""RubCloth_Phase"""


## 資料組合的問題

原本是想將量產的由 MGD 來，OC 由 combo 來補；但要由 combo 補的前提就是要核對 應用別, PI, LC, SEAL 是否一致；
如此的話所有資料由 combo 來是否即可？

In [63]:
(
    isd_attribute_db
    .select(
        pl.col("廠別").alias("使用廠區"),
        pl.col("APPLICATION").alias("應用別"),
        pl.col("資料名稱").alias("LC 型號"),
        pl.col("VOP_V").alias("Vop"),
        pl.col("TNI_C").alias("Tni"),
        pl.col("LTS_C").alias("Tcn"),
        pl.col("CELL_GAP").alias("Cell Gap"),
        pl.col("RT_MS").alias("RT"),
        pl.when(pl.col("GTG_RT") == 0).then(None).otherwise(pl.col("GTG_RT"))
        .alias("GTG"),
    )
    .filter(
        pl.col("LC 型號") == 'LCT-15-1098',
    )
)


使用廠區,應用別,LC 型號,Vop,Tni,Tcn,Cell Gap,RT,GTG
str,str,str,f64,f64,i64,f64,f64,f64
"""LCD-03""","""NB""","""LCT-15-1098""",5.5,85.1,-30,2.85,25.0,null
"""LCD-05""","""NB""","""LCT-15-1098""",5.0,85.1,-30,3.1,25.0,null
"""LCD-05""","""MP""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null
"""LCD-05""","""Tablet""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null
"""LCD-T1""","""Tablet""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null
…,…,…,…,…,…,…,…,…
"""LCD-T2""","""Tablet""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null
"""LCD-T2""","""MP""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null
"""LCD-T3""","""MP""","""LCT-15-1098""",5.5,85.1,-30,3.1,25.0,null


In [75]:
(
    combo_db
    .select(
        # "COMBOID",
        pl.col("LCD_FAB").alias("使用廠區"),
        pl.col("LC_MODE").alias("LC Mode"),
        pl.col("APPLICATION").alias("應用別"),
        pl.col("ITM_000050_DATA").alias("OC 型號"),
        pl.col("ITM_000050_PHASE").alias("OC Phase"),
        pl.col("ITM_000061_DATA").alias("PI 型號"),
        pl.col("ITM_000061_PHASE").alias("PI Phase"),
        pl.col("ITM_000064_DATA").alias("LC 型號"),
        pl.col("ITM_000064_PHASE").alias("LC Phase"),
        pl.col("ITM_000071_DATA").alias("SEAL 型號"),
        pl.col("ITM_000071_PHASE").alias("SEAL Phase"),
    )
    .unique(maintain_order=True)
    .join(
        isd_attribute_db
        .select(
            pl.col("廠別").alias("使用廠區"),
            pl.col("APPLICATION").alias("應用別"),
            pl.col("正負型液晶").alias("Type (pLC/nLC)"),
            pl.col("資料名稱").alias("LC 型號"),
            pl.col("VOP_V").alias("Vop"),
            pl.col("TNI_C").alias("Tni"),
            pl.col("LTS_C").alias("Tcn"),
            pl.col("CELL_GAP").alias("Cell Gap"),
            pl.col("RT_MS").alias("RT"),
            pl.when(pl.col("GTG_RT") == 0).then(None).otherwise(pl.col("GTG_RT"))
            .alias("GTG"),
        ),
        on=['使用廠區', '應用別', 'LC 型號'],
    )
    .select(
        "使用廠區",
        "LC Mode",
        "應用別",
        "Type (pLC/nLC)",
        "OC 型號",
        "OC Phase",
        "PI 型號",
        "PI Phase",
        "LC 型號",
        "LC Phase",
        "SEAL 型號",
        "SEAL Phase",
        "Vop",
        "Tni",
        "Tcn",
        "Cell Gap",
        "RT",
        "GTG",
    )
).write_excel('demo_20250918.xlsx')

In [74]:
(
    combo_db
    .select(
        # "COMBOID",
        pl.col("LCD_FAB").alias("使用廠區"),
        pl.col("LC_MODE").alias("LC Mode"),
        pl.col("APPLICATION").alias("應用別"),
        pl.col("ITM_000050_DATA").alias("OC 型號"),
        pl.col("ITM_000050_PHASE").alias("OC Phase"),
        pl.col("ITM_000061_DATA").alias("PI 型號"),
        pl.col("ITM_000061_PHASE").alias("PI Phase"),
        pl.col("ITM_000064_DATA").alias("LC 型號"),
        pl.col("ITM_000064_PHASE").alias("LC Phase"),
        pl.col("ITM_000071_DATA").alias("SEAL 型號"),
        pl.col("ITM_000071_PHASE").alias("SEAL Phase"),
    )
    .unique(maintain_order=True)
    .join(
        isd_attribute_db
        .select(
            pl.col("廠別").alias("使用廠區"),
            pl.col("APPLICATION").alias("應用別"),
            pl.col("正負型液晶").alias("Type (pLC/nLC)"),
            pl.col("資料名稱").alias("LC 型號"),
            pl.col("VOP_V").alias("Vop"),
            pl.col("TNI_C").alias("Tni"),
            pl.col("LTS_C").alias("Tcn"),
            pl.col("CELL_GAP").alias("Cell Gap"),
            pl.col("RT_MS").alias("RT"),
            pl.when(pl.col("GTG_RT") == 0).then(None).otherwise(pl.col("GTG_RT"))
            .alias("GTG"),
        ),
        on=['使用廠區', '應用別', 'LC 型號'],
    )
    .select(
        "使用廠區",
        "LC Mode",
        "應用別",
        "Type (pLC/nLC)",
        "OC 型號",
        "OC Phase",
        "PI 型號",
        "PI Phase",
        "LC 型號",
        "LC Phase",
        "SEAL 型號",
        "SEAL Phase",
        "Vop",
        "Tni",
        "Tcn",
        "Cell Gap",
        "RT",
        "GTG",
    )
)

使用廠區,LC Mode,應用別,Type (pLC/nLC),OC 型號,OC Phase,PI 型號,PI Phase,LC 型號,LC Phase,SEAL 型號,SEAL Phase,Vop,Tni,Tcn,Cell Gap,RT,GTG
str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,i64,f64,f64,f64
"""LCD-07""","""NPVA""","""TV""","""nLC""","""NOT_USE""","""RELEASED""","""AL65308-R1""","""RELEASED""","""LCT-12-1667""","""RELEASED""","""W/R 723K1M""","""RELEASED""",8.0,75.2,-20,3.5,16.0,9.0
"""LCD-07""","""NPVA""","""TV""","""nLC""","""NOT_USE""","""RELEASED""","""AL67001""","""RELEASED""","""LCT-12-1667""","""RELEASED""","""W/R 723K1M""","""RELEASED""",8.0,75.2,-20,3.5,16.0,9.0
"""LCD-8B""","""NPVA""","""TV""","""nLC""","""NOT_USE""","""RELEASED""","""AL67105""","""RELEASED""","""LCT-12-1667""","""RELEASED""","""W/R 723K1M""","""RELEASED""",8.0,75.2,-20,3.5,16.0,9.0
"""LCD-04""","""PA_AAS""","""MNT""","""pLC""","""SS3727-R24SLBQ""","""RELEASED""","""NRB-W185(068A)_RB089""","""TR3""","""BHR92800""","""RELEASED""","""W/R 723K1M""","""RELEASED""",6.0,73.9,-20,3.2,14.0,null
"""LCD-04""","""AAS""","""MNT""","""pLC""","""SS3727-R24SLB""","""RELEASED""","""SE-6414""","""RELEASED""","""BHR92800""","""RELEASED""","""W/R 723K1M""","""RELEASED""",6.0,73.9,-20,3.2,14.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""LCD-T1""","""PA_AAS""","""NB""","""nLC""","""TG2269-SA2""","""TR3""","""AL7171""","""TR3""","""ZIX-7061XX""","""TR3""","""S-WB141(318B)""","""TR3""",6.0,75.0,-20,3.3,25.0,22.0
"""LCD-03""","""AAS""","""NB""","""pLC""","""NOT_USE""","""RELEASED""","""SE-6414""","""RELEASED""","""ZCM-5617XX""","""TR3""","""W/R 723K1M""","""RELEASED""",10.0,116.0,-20,5.0,30.0,null
"""LCD-03""","""AAS""","""IA""","""pLC""","""SS3727-R24SLBQ""","""RELEASED""","""SE-6414""","""RELEASED""","""ZCM-5581XX""","""TR4""","""W/R 723K1M""","""RELEASED""",6.0,100.0,-40,3.2,25.0,null


In [73]:
combo_db

COMBOID,STATE_KEY,MASKID,MODELNAME,ITEMS_PHASE_KEY,APPLICATION,CF_FAB,LCD_FAB,LC_MODE,ITM_000061_DATA,ITM_000061_PHASE,ITM_000064_DATA,ITM_000064_PHASE,ITM_000050_DATA,ITM_000050_PHASE,ITM_000071_DATA,ITM_000071_PHASE,ITM_000062_DATA,ITM_000062_PHASE
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""COMBOID""","""STATE_KEY""","""MASKID""","""MODELNAME""","""Overall_Phase""","""APPLICATION""","""CF_FAB""","""LCD_FAB""","""LC_MODE""","""PI型號""","""PI_Phase""","""LC型號""","""LC_Phase""","""OC型號""","""OC_Phase""","""SEAL型號""","""SEAL_Phase""","""RubCloth型號""","""RubCloth_Phase"""
"""2405_00018""","""FILED""","""R034""","""M270HCJ-PDB""","""TR3""","""MNT""","""CF-8A""","""LCD-8A""","""AAS""","""AL7171""","""TR4""","""BHR92800""","""RELEASED""","""SS3727-R24SLBQ""","""RELEASED""","""S-WB141(318B)""","""RELEASED""","""NOT_USE""","""RELEASED"""
"""2305_00001""","""FILED""","""N819 AL7171""","""M238KCA-L8B""","""TR3""","""MNT""","""CF-8A""","""LCD-8A""","""PA_AAS""","""AL7171""","""TR4""","""BHR92800""","""RELEASED""","""SS3727-R24SLBQ""","""RELEASED""","""W/R 723K1M""","""RELEASED""","""NOT_USE""","""RELEASED"""
"""2001_00003""","""FILED""","""R011""","""M270KCJ-L5B""","""TR3""","""MNT""","""CF-T2""","""LCD-T2""","""PA_IPS""","""NRB-W185(068A)_RB089""","""TR3""","""BHR92800""","""RELEASED""","""SS3727-R24SLB""","""RELEASED""","""W/R 723K1M""","""RELEASED""","""NOT_USE""","""RELEASED"""
"""2106_00002""","""FILED""","""N817""","""M238HCA-LCZ""","""TR3""","""MNT""","""CF-8A""","""LCD-8A""","""PA_AAS""","""NRB-W185(068A)_RB089""","""RELEASED""","""BHR92800""","""RELEASED""","""SS3727-R24SLBQ""","""RELEASED""","""W/R 723K1M""","""RELEASED""","""NOT_USE""","""RELEASED"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2308_00005""","""FILED""","""5050""","""AE050ND-01B""","""TR3""","""Avionics""","""CF-T3""","""LCD-T3""","""TN""","""SE-7492H""","""RELEASED""","""ZTO-5074LA""","""RELEASED""","""NOT_USE""","""RELEASED""","""W/R 723K1M""","""TR3""","""MK-0012""","""RELEASED"""
"""2206_00015""","""FILED""","""1612""","""WZ016AA-01C""","""TR3""","""CE""","""CF-T3""","""LCD-T3""","""ECB""","""SE-7492H""","""RELEASED""","""ZTO-5197LA""","""RELEASED""","""NOT_USE""","""RELEASED""","""W/R 723K1M""","""RELEASED""","""MK-0012""","""RELEASED"""
"""2112_00004""","""FILED""","""1103""","""WE011AZ-01B""","""TR4""","""CE""","""CF-T3""","""LCD-T3""","""PA_AAS""","""SE-7492H""","""RELEASED""","""ZTO-5197LA""","""RELEASED""","""TG2288-SA1""","""RELEASED""","""W/R 723K1M""","""RELEASED""","""MK-0012""","""RELEASED"""
